In [37]:
from __future__ import print_function, division

import tensorflow as tf


from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np






In [52]:
who

Activation	 Adam	 BatchNormalization	 Conv2D	 Dense	 Dropout	 Flatten	 Input	 LeakyReLU	 
Model	 Reshape	 Sequential	 UpSampling2D	 ZeroPadding2D	 build_discriminator	 build_generator	 channels	 combined	 
discriminator	 division	 freq_rows	 generator	 img	 img_shape	 latent_dim	 mnist	 np	 
optimizer	 plt	 print_function	 sys	 tf	 time_cols	 valid	 z	 


In [38]:
freq_rows = 1025
time_cols = 313
channels = 1
latent_dim = (freq_rows, time_cols, channels)
img_shape = (freq_rows, time_cols, channels)

In [48]:
def build_generator():

    model = Sequential()

    model.add(Dense(4*4*256, activation="relu", input_dim=320825))
    model.add(Reshape((256, 4 ,4)))
    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
    model.add(Conv2D(channels, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))

    print("GENERATOR")
    model.summary()

    noise = Input(shape=(320825,))
    remix = model(noise)

    return Model(noise, remix)

In [49]:
def build_discriminator():

    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    print("DISCRIMINATOR")
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

In [51]:
optimizer = Adam(0.0002, 0.5)

# Build and compile the discriminator
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

# Build the generator
generator = build_generator()

# The generator takes noise as input and generates imgs
z = Input(shape=(320825,))
img = generator(z)

# For the combined model we will only train the generator
discriminator.trainable = False

# The discriminator takes generated images as input and determines validity
valid = discriminator(img)

# The combined model  (stacked generator and discriminator)
# Trains the generator to fool the discriminator
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

DISCRIMINATOR
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_67 (Conv2D)           (None, 513, 157, 32)      320       
_________________________________________________________________
leaky_re_lu_49 (LeakyReLU)   (None, 513, 157, 32)      0         
_________________________________________________________________
dropout_49 (Dropout)         (None, 513, 157, 32)      0         
_________________________________________________________________
conv2d_68 (Conv2D)           (None, 257, 79, 64)       18496     
_________________________________________________________________
zero_padding2d_13 (ZeroPaddi (None, 258, 80, 64)       0         
_________________________________________________________________
batch_normalization_49 (Batc (None, 258, 80, 64)       256       
_________________________________________________________________
leaky_re_lu_50 (LeakyReLU)   (None, 258, 80, 64)       0      